##Retrieve AlwaysOn Availability Group topology

##Retrieve information regarding a specific Availability Group

##Retrieve Availability Group Listener information

In [ ]:
SELECT
	G.[name] AS AvailabilityGroupName,
	L.dns_name AS ListenerName,
	L.[port] AS ListenerPort,
	I.ip_address AS IPaddress,
	I.ip_subnet_mask AS SubnetMask,
	I.network_subnet_ip AS NetworkSubnet,
	I.state_desc AS ListenerState
FROM
	sys.availability_group_listeners L
		JOIN sys.availability_group_listener_ip_addresses I
			ON L.listener_id = I.listener_id
		JOIN sys.availability_groups G
			ON L.group_id = G.group_id
ORDER BY
	G.[name]

##Retrieve Availability Group redo queue

What is this?
The redo queue is the number of log blocks waiting to be applied or re-played on secondary replicas.  The larger the redo queue, the farther behind a secondary replica is.  Ideally the redo queue is as close to zero as possible.

In [ ]:
                DECLARE @RedoQueueAlert BIGINT
				DECLARE @RecoverySecondsAlert smallint
				SET @RedoQueueAlert = 102400 --in KB
				SET @RecoverySecondsAlert = 60
				SELECT 
					CAST(DB_NAME(database_id)as VARCHAR(40)) [DBname],
					Convert(VARCHAR(20),last_commit_time,22) [LastCommitTime]
					,CAST(CAST(((DATEDIFF(s,last_commit_time,GetDate()))/3600) as varchar) + ' hour(s), '
					+ CAST((DATEDIFF(s,last_commit_time,GetDate())%3600)/60 as varchar) + ' min, '
					+ CAST((DATEDIFF(s,last_commit_time,GetDate())%60) as varchar) + ' sec' as VARCHAR(30)) [TimeBehindPrimary]
					,CAST((redo_queue_size/redo_rate/60.0) as decimal(10,2)) [EstimatedRecoveryMinutes]
					,(redo_queue_size/redo_rate) [EstimatedRecoverySeconds]
						,redo_queue_size AS RedoQueueSize
						,CASE
							WHEN redo_queue_size > @RedoQueueAlert THEN 'CRITICAL'
							WHEN redo_queue_size < @RedoQueueAlert AND redo_queue_size > 1024 THEN 'WARNING'
							WHEN redo_queue_size < @RedoQueueAlert AND redo_queue_size < 1024 THEN 'GOOD'
							WHEN redo_queue_size = 0 THEN 'GOOD'
						END AS [RedoQueueState]
				FROM
					sys.dm_hadr_database_replica_states
				WHERE
					last_redone_time is not null;